In [99]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder

In [134]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [135]:
from sklearn.model_selection import GridSearchCV

In [3]:
df = pd.read_csv('sales_data.csv')

In [7]:
df.columns

Index(['date', 'channel', 'country', 'shop_name', 'sku', 'product_name',
       'brand', 'url', 'sap', 'cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6',
       'sales', 'units_sold', 'product_views', 'click_rate', 'buyer', 'order'],
      dtype='object')

In [8]:
df = df.drop(['cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6'], axis=1)

In [10]:
df = df.drop(['buyer', 'order'], axis=1)

In [12]:
df = df.drop(['country', 'url'], axis=1)

In [19]:
df.head()

,date,channel,shop_name,sku,product_name,brand,sap,sales,units_sold,product_views,click_rate
0,2019-01-01,Lazada,Lazada Retail Lactum,260013061_PH-358536442,Lactum 3+ Plain Powdered Milk Drink 2kg,Lactum,2048174.0,0.0,0,25,0.00
1,2019-01-01,Lazada,Lazada Retail Lactum,267657810_PH-379353311,Lactum 3+ Plain Powdered Milk Drink 1.6Kg with...,Lactum,0.0,0.0,10,116,8.62
2,2019-01-01,Lazada,Lazada Retail Lactum,267665603_PH-379348476,Lactum 3+ Plain Powdered Milk Drink 1.2Kg with...,Lactum,0.0,0.0,1,27,3.70
3,2019-01-01,Lazada,Lazada Retail Lactum,267669546_PH-379343854,Lactum 3+ Plain Powdered Milk Drink 150g,Lactum,2020877.0,0.0,0,62,0.00
4,2019-01-01,Lazada,Lazada Retail Lactum,DELETE,2 Packs Of Lactum 3+ Plain Powdered Milk Drink...,Lactum,0.0,0.0,17,298,5.70


## Feature Engineering - Dates

In [21]:
df.date = pd.to_datetime(df.date)

In [22]:
df.head()

,date,channel,shop_name,sku,product_name,brand,sap,sales,units_sold,product_views,click_rate
0,2019-01-01,Lazada,Lazada Retail Lactum,260013061_PH-358536442,Lactum 3+ Plain Powdered Milk Drink 2kg,Lactum,2048174.0,0.0,0,25,0.00
1,2019-01-01,Lazada,Lazada Retail Lactum,267657810_PH-379353311,Lactum 3+ Plain Powdered Milk Drink 1.6Kg with...,Lactum,0.0,0.0,10,116,8.62
2,2019-01-01,Lazada,Lazada Retail Lactum,267665603_PH-379348476,Lactum 3+ Plain Powdered Milk Drink 1.2Kg with...,Lactum,0.0,0.0,1,27,3.70
3,2019-01-01,Lazada,Lazada Retail Lactum,267669546_PH-379343854,Lactum 3+ Plain Powdered Milk Drink 150g,Lactum,2020877.0,0.0,0,62,0.00
4,2019-01-01,Lazada,Lazada Retail Lactum,DELETE,2 Packs Of Lactum 3+ Plain Powdered Milk Drink...,Lactum,0.0,0.0,17,298,5.70


In [36]:
months = df.date.dt.month
days = df.date.dt.day
day_name = df.date.dt.day_name()
is_weekend = day_name.apply(lambda x: 1 if x in ['Saturday', 'Sunday'] else 0)

In [37]:
df1 = df.copy()

In [44]:
#dates
df_days = pd.get_dummies(days, prefix='day')
df_months = pd.get_dummies(months, prefix='month')
df_dayname = pd.get_dummies(day_name)
df_weekend = pd.get_dummies(is_weekend, prefix='weekend')
df_datefeatures = pd.concat([df_months, df_days, df_dayname, df_weekend], axis=1)

In [50]:
#channel
channel = pd.get_dummies(df1['channel'])

In [55]:
#categorical binning
df1.shop_name.unique()

array(['Lazada Retail Lactum', 'Lazada Retail Enfagrow',
       'Lazada Retail Sustagen', 'Lactum Official Store',
       'Enfagrow A+ Four Official Store', 'Sustagen Official Store',
       'LazMart Philippines', 'lactumofficialstoreph',
       'Enfagrow Official Store_PH', 'sustagenofficialstoreph',
       ' Lactum Official Store', ' Enfagrow Official Store'], dtype=object)

In [77]:
shop_conditions = [
    df1['shop_name'].str.contains('Lactum', 'lactum'),
    df1['shop_name'].str.contains('Enfagrow'),
    df1['shop_name'].str.contains('Sustagen', 'sustagen')
]

shops = ['Lactum Store', 'Enfagrow Store', 'Sustagen Store']

shop_bins = np.select(shop_conditions, shops, default='Other')


In [78]:
df1['shop_name'] = shop_bins

In [82]:
df1 = df1.drop('sku', axis=1)

In [86]:
df1 = df1.drop('product_name', axis=1)

In [88]:
df1 = df1.drop('brand', axis=1)

In [91]:
df1 = df1.drop('date', axis=1)

In [93]:
df1.head()

,channel,shop_name,sap,sales,units_sold,product_views,click_rate
0,Lazada,Lactum Store,2048174.0,0.0,0,25,0.00
1,Lazada,Lactum Store,0.0,0.0,10,116,8.62
2,Lazada,Lactum Store,0.0,0.0,1,27,3.70
3,Lazada,Lactum Store,2020877.0,0.0,0,62,0.00
4,Lazada,Lactum Store,0.0,0.0,17,298,5.70


In [125]:
df2 = df1.copy()

In [126]:
df2 = pd.concat([df_datefeatures, channel, df2], axis=1)

In [127]:
shops = pd.get_dummies(df2[['channel', 'shop_name']])

In [128]:
df2 = df2.drop(['channel', 'shop_name'], axis=1)

In [129]:
df2 = pd.concat([df2, shops], axis=1)

In [130]:
df_target = df2['sales']
df_features = df2.drop('sales', axis=1)

In [131]:
X_train_0_1, X_test_0_1, y_train_0_1, y_test_0_1 = train_test_split(
    df_features, df_target, test_size=0.25)

In [138]:
#kNN = KNeighborsRegressor(n_neighbors=8) 
kNN = KNeighborsRegressor() 
kNN.fit(X_train_0_1,y_train_0_1)
print('Fitting KNN')
#DT = DecisionTreeRegressor(max_depth=5) 
DT = DecisionTreeRegressor() 
DT.fit(X_train_0_1,y_train_0_1)
print('Fitting DT')
#RF = RandomForestRegressor(n_estimators = 100) 
RF = RandomForestRegressor() 
RF.fit(X_train_0_1,y_train_0_1)
print('Fitting RF')
#GBM = GradientBoostingRegressor(max_depth =5, learning_rate=.1) 
GBM = GradientBoostingRegressor() 
GBM.fit(X_train_0_1,y_train_0_1)
print('Fitting GBM')

# y_test_LR = LR.predict(X_test)
# y_test_Lasso = Lasso.predict(X_test)
# y_test_Ridge = Ridge1.predict(X_test)
# y_test_kNN = kNN.predict(X_test)
# y_test_DT = DT.predict(X_test)
# y_test_RF = RF.predict(X_test)
# y_test_GBM = GBM.predict(X_test)


cols = ['Regression Method','Train Accuracy', 'Test Accuracy']
df_0_1 = pd.DataFrame(columns=cols)

df_0_1.loc[0] = ['Linear Regression', LR.score(X_train_0_1, y_train_0_1), LR.score(X_test_0_1, y_test_0_1)]
df_0_1.loc[1] = ['Linear Regression + Lasso', Lasso1.score(X_train_0_1, y_train_0_1), Lasso1.score(X_test_0_1, y_test_0_1)]
df_0_1.loc[2] = ['Linear Regression + Ridge', Ridge1.score(X_train_0_1, y_train_0_1), Ridge1.score(X_test_0_1, y_test_0_1)]
df_0_1.loc[3] = ['kNN', kNN.score(X_train_0_1, y_train_0_1), kNN.score(X_test_0_1, y_test_0_1)]
df_0_1.loc[4] = ['Decision Tree',DT.score(X_train_0_1, y_train_0_1), DT.score(X_test_0_1, y_test_0_1)]
df_0_1.loc[5] = ['Random Forest',RF.score(X_train_0_1, y_train_0_1), RF.score(X_test_0_1, y_test_0_1)]
df_0_1.loc[6] = ['Gradient Boosting Method',GBM.score(X_train_0_1, y_train_0_1), GBM.score(X_test_0_1, y_test_0_1)]

df_0_1

Fitting KNN
Fitting DT
Fitting RF
Fitting GBM


,Regression Method,Train Accuracy,Test Accuracy
0,Linear Regression,0.366934,0.119027
1,Linear Regression + Lasso,0.366672,0.119095
2,Linear Regression + Ridge,0.366938,0.119086
3,kNN,0.630728,0.488919
4,Decision Tree,0.992981,0.151916
5,Random Forest,0.916078,0.510718
6,Gradient Boosting Method,0.923403,0.552112


In [139]:
cols = ['Regression Method','Train Accuracy', 'Test Accuracy']
df_0_1 = pd.DataFrame(columns=cols)

df_0_1.loc[3] = ['kNN', kNN.score(X_train_0_1, y_train_0_1), kNN.score(X_test_0_1, y_test_0_1)]
df_0_1.loc[4] = ['Decision Tree',DT.score(X_train_0_1, y_train_0_1), DT.score(X_test_0_1, y_test_0_1)]
df_0_1.loc[5] = ['Random Forest',RF.score(X_train_0_1, y_train_0_1), RF.score(X_test_0_1, y_test_0_1)]
df_0_1.loc[6] = ['Gradient Boosting Method',GBM.score(X_train_0_1, y_train_0_1), GBM.score(X_test_0_1, y_test_0_1)]

df_0_1

,Regression Method,Train Accuracy,Test Accuracy
3,kNN,0.630728,0.488919
4,Decision Tree,0.992981,0.151916
5,Random Forest,0.916078,0.510718
6,Gradient Boosting Method,0.923403,0.552112


In [140]:
est = GradientBoostingRegressor()
param_grids = {'learning_rate': [.2, 0.1, 0.05, 0.02, 0.01],
              'max_depth': [3, 4, 6, 10, 14],
              'min_samples_leaf': [2, 3, 4],
              'max_features': [.5,.3, .2] 
}   
gs_cv = GridSearchCV(est, param_grids).fit(X_train_0_1, y_train_0_1)
print(gs_cv.best_params_)

{'learning_rate': 0.2, 'max_depth': 10, 'max_features': 0.2, 'min_samples_leaf': 2}


In [144]:
GBM = GradientBoostingRegressor(learning_rate= 0.2, max_depth= 10, max_features= 0.2, min_samples_leaf= 2) 
GBM.fit(X_train_0_1,y_train_0_1)

GradientBoostingRegressor(learning_rate=0.2, max_depth=10, max_features=0.2,
                          min_samples_leaf=2)

In [145]:
print(GBM.score(X_train_0_1, y_train_0_1), GBM.score(X_test_0_1, y_test_0_1))

0.9900389189508423 0.5175830136528643
